In [255]:
def package_query(pname):
    package_query = " = g.V().has('ecosystem', 'maven').has('name', '"\
                    + pname + "').tryNext().orElseGet{graph.addVertex('ecosystem', 'maven', 'name', '"\
                    + pname + "', 'vertex_label', 'Package')};"
    return package_query

In [256]:
with open("/home/smasud/Downloads/sarah_vertx.json", "r") as f:
    data = json.load(f)

In [257]:
for package, topics in package_topics.items():
    str_gremlin_dsl = ""
    pck_query = ""
    topic_query = ""
    pname = package.strip()
    pck = pattern_2_save.sub('', pname)
    pck_query = pck + package_query(pname)
    topic_query = " ".join(map(lambda x: pck + ".property('topics', '" + x + "');", topics))
    pck_query += topic_query
    str_gremlin_dsl += pck_query
    fire_gremlin(target_gremlin_server, str_gremlin_dsl)
#     print pck_query

In [258]:
import json
import sys
import requests
from operator import itemgetter
import re
pattern = '[^a-zA-Z0-9]'
pattern_2_save = re.compile(pattern)

In [259]:
target_gremlin_server = "http://localhost:8181"

In [260]:
def fire_gremlin(gremlin_server, str_gremlin):
    payload = {'gremlin': str_gremlin}
    response = requests.post(gremlin_server, data=json.dumps(payload))
    if response.status_code != 200:
        print ("ERROR %d: %s") % (response.status_code, response.reason)
    json_response = response.json()
    print json_response

In [261]:
def get_packages_from_graph(list_topics, package):
    str_topics = ','.join( map(lambda x: "'" + x + "'", list_topics) )
    payload = {'gremlin': "g.V().has('vertex_label','Package').not(has('name', '"+ package +"'))has('topics', within(" + str_topics + ")).dedup().valueMap(true);"}
    response = requests.post(target_gremlin_server, data=json.dumps(payload))
    json_response = response.json()
    packages = json_response.get('result').get('data',[])
    return packages

In [262]:
def get_topics(package):
    payload = {'gremlin': "g.V().has('vertex_label', 'Package').has('name', '" + package + "').valueMap(true);"}
    response = requests.post(target_gremlin_server, data=json.dumps(payload))
    json_response = response.json()
    pck = json_response.get('result').get('data',[])
    return pck[0].get('topics')

In [263]:
def get_common_list(output_packages, package_name, input_topics_set):
    input_topics_size = len(input_topics_set)
    result = []
    if input_topics_size==0:
            return []
    for package in output_packages:
        d = {}
        output_topics_set = set(package.get('topics'))
        common_topics = input_topics_set.intersection(output_topics_set)
        common_topics_size = len(common_topics)
        if common_topics_size<2 and input_topics_size >1:
            continue
        d['input_package'] = package_name
        d['output_package'] = package.get('name')[0]
        d['input_pck_topics'] = input_topics_set
        d['output_pck_topics'] = output_topics_set
        d['common_topics_size'] = common_topics_size
        result.append(d)
    return result

In [264]:
def get_alternate_list(result, input_stack):    
    newlist = sorted(result, key=itemgetter('common_topics_size'), reverse=True)
    alternate_list = [output['output_package'] for output in newlist if output['output_package'] not in input_stack ]
    return alternate_list

In [265]:
def get_alternate_package(input_stack):
    for package_name in input_stack:
        input_pck_topics = get_topics(package_name)
        output_packages = get_packages_from_graph(input_pck_topics, package_name)
        input_topics_set = set(input_pck_topics)
        result = get_common_list(output_packages, package_name, input_topics_set)
        alternate_list = get_alternate_list(result, input_stack)
        print(alternate_list)

In [266]:
input_pck_name = ['io.vertx:vertx-mongo-embedded-db']
output = get_alternate_package(input_pck_name)
output

[]


In [267]:
input_stack = ['io.netty:netty-buffer']
get_alternate_package(input_stack)

[u'io.netty:netty-all']


In [268]:
input_stack = ['io.knotx:knotx-core']
get_alternate_package(input_stack)

[u'io.knotx:knotx-knot-fragment-splitter', u'io.knotx:knotx-mocks', u'io.knotx:knotx-example-gateway', u'io.knotx:knotx-knot-action', u'io.vertx:vertx-mqtt-server::sources', u'io.vertx:vertx-mqtt-server:zip:docs', u'io.vertx:vertx-circuit-breaker:zip:docs', u'io.vertx:vertx-hazelcast', u'io.vertx:vertx-circuit-breaker::sources', u'io.engagingspaces:vertx-dataloader', u'io.vertx.blueprint:account-microservice', u'io.vertx:vertx-core::sources']


In [271]:
input_stack = ['p1', 'p2']
get_alternate_package(input_stack)

[u'p4', u'p3']
[u'p4']


In [270]:
# """
# p1: t1, t2, t3

# p2: t2

# p3: t1, t3

# p4: t2,t3
# """

# inputstack= [p1, p2]